<a href="https://colab.research.google.com/github/Filippo-Tombari/PdeGraph/blob/main/Obstacle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the necessary packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/MyDrive/tesi/PdeGraph")
import os
os.chdir("/content/drive/MyDrive/tesi/PdeGraph")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import install
install.pytorchgeo()

Pytorch geometric installed.


In [3]:
install.fenics()

FEniCS installed.


In [4]:
import numpy as np
import pickle
import functional
from functional import asfield, plot, L2, buildconnectivity
import gnns
import dolfin
import torch
import torch.nn.functional as F
from torch_geometric.loader import NeighborSampler
import torch.optim as optim

# Loading and preparation of the data

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
mesh_train = dolfin.cpp.mesh.Mesh("files/obstacleDFG.xml")
edge_index_train = buildconnectivity(mesh_train)
edge_index_train = torch.t(torch.from_numpy(edge_index_train.astype('int32')).long()).to(device)

Training data parameter: 1.5

In [7]:
with open("/content/drive/MyDrive/tesi/obstacle/files/train_set.pkl", "rb") as fp:   #Pickling
  train_set_new = pickle.load(fp)
  train_set_new = [data.float() for data in train_set_new]
#choose a specific time window for the training set in order to capture the phenomenon that we want to predict
train_set = train_set_new[200:300] # we choose the window [4s,6s]
train_set = torch.stack(train_set, axis = 2).to(device) #now the training set has size sample size x nodes x features

# Training

In [8]:
gnns.RecurrentGNN(node_features=2,filters=32)

RecurrentGNN(
  (recurrent): GConvGRU(
    (conv_x_z): ChebConv(2, 32, K=3, normalization=sym)
    (conv_h_z): ChebConv(32, 32, K=3, normalization=sym)
    (conv_x_r): ChebConv(2, 32, K=3, normalization=sym)
    (conv_h_r): ChebConv(32, 32, K=3, normalization=sym)
    (conv_x_h): ChebConv(2, 32, K=3, normalization=sym)
    (conv_h_h): ChebConv(32, 32, K=3, normalization=sym)
  )
  (linear): Linear(in_features=32, out_features=2, bias=True)
)

In [9]:
l2   = L2(mesh_train).float() # L2 norm for scalar functions
lv22 = lambda v: l2(v[:,0].unsqueeze(dim = 0)).pow(2).float() + l2(v[:,1].unsqueeze(dim = 0)).pow(2).float()
lv2  = lambda v: lv22(v).sqrt().float() # L2 norm for vectorial functions
def loss(output, target):
  return (lv2(target - output) / lv2(target)).mean().float()

In [12]:
# use 3 snapshots to predict the following one
gnn = gnns.RecurrentGNN(node_features=2,filters=32).to(device)
valid_size = 25
train_size = train_set.shape[2] - valid_size
train = train_set[:,:,:train_size].to(device)
valid = train_set[:,:,train_size:].to(device)
snapshot_size = 1
batch_size = 25
learningrate = 1e-2
optimizer = optim.Adam(gnn.parameters(), lr=learningrate)
#optimizer = optim.LBFGS(gnn.parameters())
model_chk_path = 'checkpoints/obstacle_chk.pt'
rollout_train_out = [train[:,:,0]]
mse_min = 10000
early_stopping = 0
epochs = 100 
t = 1 # current epoch
done = False
gnn.train()
while not done:
      rollout_train_out = [train[:,:,0]]
      train_loss = 0
      step = 0
      for i in range(train_size-1):
        # training
        optimizer.zero_grad()
        #def closure():
        # forward pass
        train_out = gnn.forward(train[:,:,i], edge_index_train)
        #print(train_out.shape)
        rollout_train_out.append(train_out)
        train_loss =  loss(train_out,train[:,:,i+1])
        train_loss.backward()
        optimizer.step()
        #step += 1
        #return train_loss
      #train_loss/=(step + 1)
      # backpropagation
      
      
      print("Epoch {} train MSE: {:.4f}".format(t, train_loss.item()))
      
      if train_loss < mse_min:
        mse_min = train_loss
        valid_out_best = rollout_train_out
        torch.save(gnn, model_chk_path)
        early_stopping = 0
        print('Saving model checkpoint')
      else:
        early_stopping += 1
      #stop the training after reaching the number of epochs
      t += 1
      #import pdb; pdb.set_trace()
      if (t > epochs ): #or early_stopping == 20
        done = True

Epoch 1 train MSE: 0.1002
Saving model checkpoint
Epoch 2 train MSE: 0.0911
Saving model checkpoint
Epoch 3 train MSE: 0.0903
Saving model checkpoint
Epoch 4 train MSE: 0.0899
Saving model checkpoint
Epoch 5 train MSE: 0.0898
Saving model checkpoint
Epoch 6 train MSE: 0.0896
Saving model checkpoint
Epoch 7 train MSE: 0.0895
Saving model checkpoint
Epoch 8 train MSE: 0.0894
Saving model checkpoint
Epoch 9 train MSE: 0.0894
Saving model checkpoint
Epoch 10 train MSE: 0.0893
Saving model checkpoint
Epoch 11 train MSE: 0.0892
Saving model checkpoint
Epoch 12 train MSE: 0.0891
Saving model checkpoint
Epoch 13 train MSE: 0.0891
Saving model checkpoint
Epoch 14 train MSE: 0.0890
Saving model checkpoint
Epoch 15 train MSE: 0.0890
Saving model checkpoint
Epoch 16 train MSE: 0.0889
Saving model checkpoint


KeyboardInterrupt: ignored

In [19]:
gnn.eval()
rollout_valid = [valid[:,:,0]]
step = 0
valid_loss = 0
for i in range(valid_size - 1):
  valid_out = gnn.forward(valid[:,:,i], edge_index_train)
  rollout_valid.append(valid_out)
  valid_loss +=  loss(valid_out,valid[:,:,i+1])
  step += 1

valid_loss /= step
print("Test MSE: {:.4f}".format(valid_loss.item())) 


Test MSE: 0.0906


In [16]:
# Righe di codice per salvare l'animazione in formato .gif
import imageio
import matplotlib.pyplot as plt
def savegif(drawframe, frames, name, transparency = False, remove = True):
    filenames = []
    for i in range(frames):
        # plot frame
        drawframe(i)

        # create file name and append it to a list
        filename = f'{i}.png'
        filenames.append(filename)

        # save frame
        plt.savefig(filename, transparency = transparency)
        plt.close()
    # build gif
    with imageio.get_writer(name + '.gif', mode='I') as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Remove files
    if(remove):
        for filename in set(filenames):
            os.remove(filename)

def trajectorytogif(traj, dt, name):
    def drawframe(i):
        colorbar = plot(asfield(traj[i][:,0:2].detach().cpu().numpy(), mesh_train))
        plt.colorbar(colorbar, shrink = 0.75)
        plt.title("T = %.2f" % (dt*i))
        plt.axis("off")
    savegif(drawframe, frames = len(traj), name = name)

In [20]:
trajectorytogif(rollout_valid, 1, name = "images/prediction_rnn") # crea e salva la gif (la si trova nella cartella dei file generati, a sx del notebook)

# Nota: su Colab non si può, ma su jupyter notebook è invece possibile visualizzare poi la gif direttamente
# dentro il notebook, e.g.

# from  IPython.display import Image as show
# show("esempio.gif")

In [ ]:
for i in range(valid_size):
  plt.figure(figsize = (12,4))
  plt.subplot(1,2,1)
  plt.title("prediction")
  plot(asfield(valid_out_best[i][:,0:2].detach().numpy()))
  plt.subplot(1,2,2)
  plt.title(f"T = {250 + i}")
  plot(asfield(train_set[50 + i][:,0:2].detach().numpy()))

In [14]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Obstacle.ipynb
	modified:   __pycache__/gnns.cpython-37.pyc
	modified:   __pycache__/install.cpython-37.pyc
	modified:   checkpoints/obstacle_chk.pt
	modified:   gnns.py
	modified:   install.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Obstacle_temporal.ipynb
	images/best_prediction_1.gif

no changes added to commit (use "git add" and/or "git commit -a")


In [21]:
!git config --global user.email "filo.tombari@gmail.com"
!git config --global user.name "Filippo-Tombari"

In [ ]:
!git add .
!git commit -m "aggiunto training con GRNN"
!git push -u origin main

[main cdf140a] training con gpu e lbfgs
 6 files changed, 5 insertions(+), 3 deletions(-)
 rewrite Obstacle.ipynb (98%)
 create mode 100644 __pycache__/functional.cpython-37.pyc
 create mode 100644 __pycache__/gnns.cpython-37.pyc
 create mode 100644 __pycache__/install.cpython-37.pyc
 create mode 100644 checkpoints/obstacle_chk.pt
Counting objects: 10, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (10/10), done.
Writing objects: 100% (10/10), 505.71 KiB | 12.97 MiB/s, done.
Total 10 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/Filippo-Tombari/PdeGraph.git
   e76a7c6..cdf140a  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.
